In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import time
import datetime

df = pd.read_csv("https://raw.githubusercontent.com/daniel-dc-cd/data_science/master/module_4_ML/data/seattle_weather_1948-2017.csv")

numrows = 25549

In [52]:
heuristic_df = pd.DataFrame({'yesterday':[0.0]*numrows,
                             'today':[0.0]*numrows,
                             'tomorrow':[0.0]*numrows,
                             'guess':[False]*numrows, #logical guess
                             'rain_tomorrow':[False]*numrows, #historical observation
                             'correct':[False]*numrows, #TRUE if your guess matches the historical observation
                             'true_positive':[False]*numrows, #TRUE If you said it would rain and it did
                             'false_positive':[False]*numrows,#TRUE If you sait id would rain and it didn't
                             'true_negative':[False]*numrows, #TRUE if you said it wouldn't rain and it didn't
                             'false_negative':[False]*numrows}) #TRUE if you said it wouldn't raing and it did

#sort columns for convience
seq = ['yesterday',
       'today',
       'tomorrow',
       'guess',
       'rain_tomorrow',
       'correct',
       'true_positive',
       'false_positive',
       'true_negative',
       'false_negative']
heuristic_df = heuristic_df.reindex(columns=seq)

In [53]:
for z in range(numrows):
    #start at time 2 in the data frame
    i = z + 2
    #pull values from the dataframe
    yesterday = df.iloc[(i-2),1]
    today = df.iloc[(i-1),1]
    tomorrow = df.iloc[i,1]
    rain_tomorrow = df.iloc[(i),1]
    
    heuristic_df.iat[z,0] = yesterday
    heuristic_df.iat[z,1] = today
    heuristic_df.iat[z,2] = tomorrow
    heuristic_df.iat[z,3] = False # set guess default to False
    heuristic_df.iat[z,4] = rain_tomorrow
    
   
    if today > 0.0 and yesterday > 0.0:
        heuristic_df.iat[z,3] = True
        
    if heuristic_df.iat[z,3] == heuristic_df.iat[z,4]:
        heuristic_df.iat[z,5] = True
        if heuristic_df.iat[z,3] == True:
            heuristic_df.iat[z,6] = True #true positive
        else:
            heuristic_df.iat[z,8] = True #true negative
    else:
        heuristic_df.iat[z,5] = False
        if heuristic_df.iat[z,3] == True:
            heuristic_df.iat[z,7] = True #false positive
        else:
            heuristic_df.iat[z,9] = True #false negative

In [54]:
#Seperate a dataset into training and testing subsets

from sklearn.model_selection import train_test_split 

X=heuristic_df
y=heuristic_df
y=y.astype('str')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

X_train

,yesterday,today,tomorrow,guess,rain_tomorrow,correct,true_positive,false_positive,true_negative,false_negative
4423,0.00,0.00,0.42,False,True,False,False,False,False,True
15779,0.00,0.00,0.00,False,False,True,False,False,True,False
674,0.00,0.00,0.01,False,True,False,False,False,False,True
13864,0.00,0.00,0.00,False,False,True,False,False,True,False
21470,0.00,0.04,0.43,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...
19157,0.00,0.00,0.00,False,False,True,False,False,True,False
21813,0.01,0.00,0.00,False,False,True,False,False,True,False
9750,0.00,0.00,0.00,False,False,True,False,False,True,False
42,0.00,0.00,0.50,False,True,False,False,False,False,True


In [55]:
X_test

,yesterday,today,tomorrow,guess,rain_tomorrow,correct,true_positive,false_positive,true_negative,false_negative
16575,0.26,0.14,0.12,True,True,True,True,False,False,False
17441,0.42,0.15,0.00,True,False,False,False,True,False,False
3960,0.12,0.52,0.20,True,True,True,True,False,False,False
2828,0.00,0.00,0.00,False,False,True,False,False,True,False
21099,0.05,0.06,0.01,True,True,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...
2228,0.00,0.00,0.00,False,False,True,False,False,True,False
2087,0.00,0.00,0.00,False,False,True,False,False,True,False
9124,0.24,1.00,0.89,True,True,True,True,False,False,False
20234,0.02,0.00,0.00,False,False,True,False,False,True,False


In [56]:
#Calculate Precision for training and test sets


# for training set
p= X_train['true_positive'].sum() / (X_train['true_positive'].sum() +
                                       X_train['false_positive'].sum())
print("Training Recall:", p)

# for testing set
pr=X_test['true_positive'].sum() / (X_test['true_positive'].sum() + 
                                       X_test['false_negative'].sum())
print("Testing Recall:", pr)

Training Recall: 0.6774706701133426
Testing Recall: 0.4505732878834831


In [57]:
# Calculate Recall for training and test sets

#  training set
F=X_train['false_negative'].sum()
re=p/(p+F)
print("Training Recall:", rec)

# for testing set
N=X_test['false_negative'].sum()
re=pr/(pr+N)
print("Testing Recall:", rec)


Training Recall: 0.0003644806724019823
Testing Recall: 0.0003644806724019823


In [58]:
# Calculate SSE for both training and test sets


import numpy as np
from sklearn.metrics import mean_squared_error

def sse(y_true, y_pred):
    '''returns sum of squared errors (actual vs model)'''
    squared_errors = (y_true - y_pred) ** 2
    return np.sum(squared_errors)

# training set
SSE_tr = sse(y_true=X_train['rain_tomorrow'].astype('int'), y_pred=X_train['guess'].astype('int'))
mean_SSE_tr = mean_squared_error(y_true=X_train['rain_tomorrow'].astype('int'), y_pred=X_train['guess'].astype('int'))
print("Training SSE:" ,SSE_tr)
print("Training Mean SSE:" ,mean_SSE_tr)

# testing set
SSE_te = sse(y_true=X_test['rain_tomorrow'].astype('int'), y_pred=X_test['guess'].astype('int'))
mean_SSE_te = mean_squared_error(y_true=X_test['rain_tomorrow'].astype('int'), y_pred=X_test['guess'].astype('int'))
print("Testing SSE:" ,SSE_te)
print("Testing Mean SSE:" ,mean_SSE_te)

Training SSE: 5889
Training Mean SSE: 0.3292887497204205
Testing SSE: 2501
Testing Mean SSE: 0.32628832354859755
